test 是干扰模型，test_class是分类模型
测试结果：
intel 异步结果：

model|FPS
:-:|:-:
model1|378
model2|106

我们自己的同步：

model|FPS
:-:|:-:
model1|207
model2|40.5

我们的异步：

model1:
infer request=6
FPS=274
model2:
infer request=4
FPS=74

In [2]:
import sys
#from importlib import reload  # Not needed in Python 2
import logging as log
#reload(logging)
log.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=log.INFO,stream=sys.stdout)
log.info('Hello world!')

2020-01-13 11:03:47,540 INFO:Hello world!


In [7]:
import tensorflow as tf
import os
import numpy as np
import shutil
import tensorflow.contrib.slim as slim
from tensorflow.contrib.layers.python.layers import utils 


In [11]:
tf.reset_default_graph()

@slim.add_arg_scope
def block(inputs, fir, sec, thir, four, name, outputs_collections = None):
    with tf.variable_scope(name) as scope:
        branch_1x3 = slim.conv2d(inputs, fir, [1, 5], [1, 1], padding = "SAME", scope = 'branch_1x5')
        branch_1x7 = slim.conv2d(inputs, sec, [1, 9], [1, 1], padding = "SAME", scope = 'branch_1x9')
        branch_1x11 = slim.conv2d(inputs, thir, [1, 21], [1, 1], padding = "SAME", scope = 'branch_1x21')

        branch_poola = slim.avg_pool2d(inputs, [1, 5], [1, 1], padding = "SAME", scope = 'branch_poola')
        branch_poolb = slim.conv2d(branch_poola, four, [1, 1], [1, 1], padding = "SAME", scope = 'branch_poolb')
        out = tf.concat(axis = 3, values = [branch_1x3, branch_1x7, branch_1x11, branch_poolb])
        out = slim.utils.collect_named_outputs(outputs_collections, scope.name, out)

        return out
def inceptionv1_arg_scope(weight_decay = 0.0005):
    with slim.arg_scope([slim.variable, slim.model_variable], device = '/cpu:0'):
        with slim.arg_scope(
                [slim.conv2d, slim.fully_connected],
                weights_initializer = slim.variance_scaling_initializer(),
                weights_regularizer = slim.l2_regularizer(weight_decay),
                activation_fn = tf.nn.relu,
                biases_initializer = tf.zeros_initializer()):
            with slim.arg_scope([slim.conv2d], padding = 'SAME'):
                with slim.arg_scope([slim.avg_pool2d], padding = 'SAME') as arg_sc:
                    return arg_sc
def noise(inputs,
          num_classes = 1000,
          dropout_keep_prob = 0.5,
          is_training=True):
    with tf.variable_scope('test_model', [inputs, num_classes]) as sc:
#         tf.reset_default_graph()
        # Collect outputs for conv2d, fully_connected and max_pool2d.
        end_points_collection = sc.original_name_scope + 'end_points'
        with slim.arg_scope(
                [slim.conv2d, slim.fully_connected, slim.max_pool2d],
                outputs_collections = end_points_collection):
            net = slim.conv2d(inputs, 128, [1, 5], [1, 1], scope = 'conv1')  # 496
            net = slim.conv2d(inputs, 128, [1, 5], [1, 1], scope = 'conv1-1')  # 496
            net = slim.max_pool2d(net, [1, 2], [1, 2], scope = 'pool1')  # 248
            net = slim.conv2d(net, 256, [1, 5], [1, 1], scope = 'conv2')  # 244
            net = slim.max_pool2d(net, [1, 2], [1, 2], scope = 'pool2')  # 122
            net = slim.conv2d(net, 256, [1, 5], [1, 1], scope = 'conv3')  # 118
            net = slim.max_pool2d(net, [1, 2], [1, 2], scope = 'pool3')  # 59
            shape = net.get_shape().as_list()
            net = tf.reshape(net, [-1, shape[1] * shape[2] * shape[3]])
            net = slim.fully_connected(net, 256, scope = 'fc1')
            net = slim.fully_connected(net, 512, scope = 'fc2')
#             net = slim.dropout(net, dropout_keep_prob, scope = 'dropout2')
            net = slim.fully_connected(net, num_classes, activation_fn = None, scope = 'logits')
            end_points = utils.convert_collection_to_dict(end_points_collection)
            if num_classes is not None:
                end_points['predictions'] = slim.softmax(net, scope = 'predictions')
            return net, end_points

    
def _activation_summary(x):

    print("name:", x.op.name, ' ', "shape:", x.get_shape().as_list())

def _activation_summaries(endpoints):
    with tf.name_scope('summaries'):
        print('____________________________model output shape_______________________________')
        for act in endpoints.values():
            _activation_summary(act)
def inference_inceptionv1(images, num_classes, is_training = False):
    with slim.arg_scope(inceptionv1_arg_scope()):
        logits, endpoints = noise(
            images,
            dropout_keep_prob = 0.5,
            num_classes = num_classes,
            is_training = is_training)

    # Add summaries for viewing model statistics on TensorBoard.
    _activation_summaries(endpoints)

    # Grab the logits associated with the side head. Employed during training.

    return logits

test='test'
model_out='test/model_out'
model_converted='test/model_converted'
if os.path.exists(test):
    t=input("The path(%s) is exist,do you want to delete it and remake?(y/n):\n" %(test,)).strip()
    while t not in ['y','n']:
        t = input("your input is invalid,please input again(y/n):\n")
    if t =='y':
        shutil.rmtree(test)
        os.makedirs(model_out)
        os.makedirs(model_converted)
    else:
        pass
else:
    print("The test path is not exist! make it !\n")
    os.makedirs(model_out)
    os.makedirs(model_converted)
       
    
with tf.Graph().as_default() as g:
    images=tf.placeholder(dtype=tf.float32,shape=(None,1,500,1),name='input_node')
    logits = inference_inceptionv1(images, 3, is_training = False)
    probs = tf.nn.softmax(logits,name='myoutputnode')
    saver=tf.train.Saver()
    tf.set_random_seed(123)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        saver.save(sess,model_out+'/model.ckpt')
        graph = tf.get_default_graph()
        input_graph_def = graph.as_graph_def()
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            input_graph_def,
            ['myoutputnode']
        )
        outgraph=tf.graph_util.remove_training_nodes(output_graph_def)
        for node in outgraph.node:
            print('node:',node.name)
        tf.train.write_graph(outgraph,model_out,'frozen_model.pb',as_text=False)
        tf.train.write_graph(outgraph,model_out,'frozen_model.pbtxt')
            

The path(test) is exist,do you want to delete it and remake?(y/n):
y
____________________________model output shape_______________________________
name: test_model/conv1/Relu   shape: [None, 1, 500, 128]
name: test_model/conv1-1/Relu   shape: [None, 1, 500, 128]
name: test_model/pool1/MaxPool   shape: [None, 1, 250, 128]
name: test_model/conv2/Relu   shape: [None, 1, 250, 256]
name: test_model/pool2/MaxPool   shape: [None, 1, 125, 256]
name: test_model/conv3/Relu   shape: [None, 1, 125, 256]
name: test_model/pool3/MaxPool   shape: [None, 1, 62, 256]
name: test_model/fc1/Relu   shape: [None, 256]
name: test_model/fc2/Relu   shape: [None, 512]
name: test_model/logits/BiasAdd   shape: [None, 3]
name: test_model/predictions/Reshape_1   shape: [None, 3]
2020-01-07 19:18:19,887 INFO:Froze 12 variables.
2020-01-07 19:18:19,907 INFO:Converted 12 variables to const ops.
node: input_node
node: test_model/conv1-1/weights
node: test_model/conv1-1/biases
node: test_model/conv1-1/Conv2D
node: test_m

In [3]:
%%bash 
source /opt/intel/openvino/bin/setupvars.sh

[setupvars.sh] OpenVINO environment initialized


In [12]:
%%bash 
scribe="/opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py"
python3 $scribe --input input_node --output myoutputnode --input_model ./test/model_out/frozen_model.pb --model_name model_fp16 --output_dir ./test/model_converted/model_fp16 --log_level=ERROR --data_type=FP16 --input_shape [1,1,500,1]

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/tianliang/2019r3/./test/model_out/frozen_model.pb
	- Path for generated IR: 	/home/tianliang/2019r3/./test/model_converted/model_fp16
	- IR output name: 	model_fp16
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	input_node
	- Output layers: 	myoutputnode
	- Input shapes: 	[1,1,500,1]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update the configuration file with input/output node names: 	None
	- Use configuration file used to generate t

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [13]:
tf.reset_default_graph()

@slim.add_arg_scope
def block(inputs, fir, sec, thir, four, name, outputs_collections = None):
    with tf.variable_scope(name) as scope:
        branch_1x3 = slim.conv2d(inputs, fir, [1, 5], [1, 1], padding = "SAME", scope = 'branch_1x5')
        branch_1x7 = slim.conv2d(inputs, sec, [1, 9], [1, 1], padding = "SAME", scope = 'branch_1x9')
        branch_1x11 = slim.conv2d(inputs, thir, [1, 21], [1, 1], padding = "SAME", scope = 'branch_1x21')

        branch_poola = slim.avg_pool2d(inputs, [1, 5], [1, 1], padding = "SAME", scope = 'branch_poola')
        branch_poolb = slim.conv2d(branch_poola, four, [1, 1], [1, 1], padding = "SAME", scope = 'branch_poolb')
        out = tf.concat(axis = 3, values = [branch_1x3, branch_1x7, branch_1x11, branch_poolb])
        out = slim.utils.collect_named_outputs(outputs_collections, scope.name, out)

        return out
def inceptionv1_arg_scope(weight_decay = 0.0005):
    with slim.arg_scope([slim.variable, slim.model_variable], device = '/cpu:0'):
        with slim.arg_scope(
                [slim.conv2d, slim.fully_connected],
                weights_initializer = slim.variance_scaling_initializer(),
                weights_regularizer = slim.l2_regularizer(weight_decay),
                activation_fn = tf.nn.relu,
                biases_initializer = tf.zeros_initializer()):
            with slim.arg_scope([slim.conv2d], padding = 'SAME'):
                with slim.arg_scope([slim.avg_pool2d], padding = 'SAME') as arg_sc:
                    return arg_sc
                
def inceptionv1(inputs,
          num_classes = 1000,
          dropout_keep_prob = 0.5,
          is_training=True):
    with tf.variable_scope('inceptionv1', [inputs, num_classes]) as sc:
        end_points_collection = sc.original_name_scope + '_end_points'
        # Collect outputs for conv2d, fully_connected and max_pool2d.

        with slim.arg_scope(
                [slim.conv2d, slim.fully_connected, slim.max_pool2d, block],
                outputs_collections = end_points_collection):
            # print(inputs.get_shape()[1].value)
            net = slim.conv2d(inputs, 64, [inputs.get_shape()[1].value, 5], [1, 2],padding="VALID", scope = 'conv1')  # 996
            net = slim.max_pool2d(net, [1, 3], [1, 2], scope = 'pool1')  # 497
            net = slim.conv2d(net, 96, [1, 5], [1, 1], scope = 'conv2')  # 493
            net = slim.max_pool2d(net, [1, 3], [1, 2], scope = 'pool2')  # 246
            net = block(net, 32, 64, 16, 16, 'mixed_128')  # 246
            net = block(net, 64, 96, 48, 32, 'mixed_240')  # 246
            net = slim.max_pool2d(net, [1, 3], [1, 2], scope = 'pool_after_mixed_240')  # 122
            net = block(net, 96, 112, 24, 32, 'mixed_264')  # 122
            net = block(net, 80, 128, 32, 32, 'mixed_272_1')  # 122
            net = slim.max_pool2d(net, [1, 3], [1, 2], scope = 'pool_after_mixed_272_1')  # 60
            net = block(net, 64, 144, 32, 32, 'mixed_272_2')  # 60
            net = block(net, 96, 160, 64, 64, 'mixed_384_1')  # 60
            net = slim.max_pool2d(net, [1, 3], [1, 2], scope = 'pool_after_mixed_384_1')  # 29
            net = block(net, 96, 160, 64, 64, 'mixed_384_2')  # 29
            net = block(net, 128, 192, 80, 80, 'mixed_480')  # 29
            net = slim.max_pool2d(net, [1, 3], [1, 2], scope = 'pool_after_mixed_288')  # 14
            shape = net.get_shape().as_list()
            print(shape)
#             net = tf.reshape(net, [-1, shape[1] * shape[2] * shape[3]])
#             net = slim.fully_connected(net, 512, scope = "fc1")
#             net = slim.fully_connected(net, 512, scope = "fc2")
            # net = slim.conv2d(net, 256, [1, 5], [1, 1], scope = 'conv_end')  # 6
            # print(net)
            net = slim.avg_pool2d(net, [1, 6], [1, 1],padding="VALID", scope = 'pool_end')  # 1
            #
            net = slim.conv2d(net, 512, [1, 1], scope = 'conv_fc1')
            net = slim.conv2d(net, 512, [1, 1], scope = 'conv_fc2')
           
            # net = slim.conv2d(net, num_classes, [1, 1], activation_fn = None, normalizer_fn = None,
            #                   scope = 'logits')
            net = slim.fully_connected(net, num_classes, activation_fn = None, scope = 'logits')
            if net.get_shape().ndims != 2:
                net = tf.squeeze(net, [1, 2], name = 'SpatialSqueeze_logits')
            end_points = utils.convert_collection_to_dict(end_points_collection)
            if num_classes is not None:
                end_points['predictions'] = slim.softmax(net, scope = 'predictions')
            return net, end_points

    
def _activation_summary(x):

    print("name:", x.op.name, ' ', "shape:", x.get_shape().as_list())

def _activation_summaries(endpoints):
    with tf.name_scope('summaries'):
        print('____________________________model output shape_______________________________')
        for act in endpoints.values():
            _activation_summary(act)
def inference_inceptionv1(images, num_classes, is_training = False):
    with slim.arg_scope(inceptionv1_arg_scope()):
        logits, endpoints = inceptionv1(
            images,
            dropout_keep_prob = 0.5,
            num_classes = num_classes,
            is_training = is_training)

    # Add summaries for viewing model statistics on TensorBoard.
    _activation_summaries(endpoints)

    # Grab the logits associated with the side head. Employed during training.

    return logits

test='test_class'
model_out='test_class/model_out'
model_converted='test_class/model_converted'
if os.path.exists(test):
    t=input("The path(%s) is exist,do you want to delete it and remake?(y/n):\n" %(test,)).strip()
    while t not in ['y','n']:
        t = input("your input is invalid,please input again(y/n):\n")
    if t =='y':
        shutil.rmtree(test)
        os.makedirs(model_out)
        os.makedirs(model_converted)
    else:
        pass
else:
    print("The test path is not exist! make it !\n")
    os.makedirs(model_out)
    os.makedirs(model_converted)
       
    
with tf.Graph().as_default() as g:
    images=tf.placeholder(dtype=tf.float32,shape=(None,3,1000,1),name='input_node')
    logits = inference_inceptionv1(images, 12, is_training = False)
    probs = tf.nn.softmax(logits,name='myoutputnode')
    saver=tf.train.Saver()
    tf.set_random_seed(123)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        saver.save(sess,model_out+'/model.ckpt')
        graph = tf.get_default_graph()
        input_graph_def = graph.as_graph_def()
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            input_graph_def,
            ['myoutputnode']
        )
        outgraph=tf.graph_util.remove_training_nodes(output_graph_def)
        for node in outgraph.node:
            print('node:',node.name)
        tf.train.write_graph(outgraph,model_out,'frozen_model.pb',as_text=False)
        tf.train.write_graph(outgraph,model_out,'frozen_model.pbtxt')
            

The path(test_class) is exist,do you want to delete it and remake?(y/n):
y
[None, 1, 6, 480]
____________________________model output shape_______________________________
name: inceptionv1/conv1/Relu   shape: [None, 1, 498, 64]
name: inceptionv1/pool1/MaxPool   shape: [None, 1, 248, 64]
name: inceptionv1/conv2/Relu   shape: [None, 1, 248, 96]
name: inceptionv1/pool2/MaxPool   shape: [None, 1, 123, 96]
name: inceptionv1/mixed_128/branch_1x5/Relu   shape: [None, 1, 123, 32]
name: inceptionv1/mixed_128/branch_1x9/Relu   shape: [None, 1, 123, 64]
name: inceptionv1/mixed_128/branch_1x21/Relu   shape: [None, 1, 123, 16]
name: inceptionv1/mixed_128/branch_poolb/Relu   shape: [None, 1, 123, 16]
name: inceptionv1/mixed_128/concat   shape: [None, 1, 123, 128]
name: inceptionv1/mixed_240/branch_1x5/Relu   shape: [None, 1, 123, 64]
name: inceptionv1/mixed_240/branch_1x9/Relu   shape: [None, 1, 123, 96]
name: inceptionv1/mixed_240/branch_1x21/Relu   shape: [None, 1, 123, 48]
name: inceptionv1/mixed

In [8]:
tf.reset_default_graph()

@slim.add_arg_scope
def block(inputs, fir, sec, thir, four, name, outputs_collections = None):
    with tf.variable_scope(name) as scope:
        branch_1x3 = slim.conv2d(inputs, fir, [1, 5], [1, 1], padding = "SAME", scope = 'branch_1x5')
        branch_1x7 = slim.conv2d(inputs, sec, [1, 9], [1, 1], padding = "SAME", scope = 'branch_1x9')
        branch_1x11 = slim.conv2d(inputs, thir, [1, 21], [1, 1], padding = "SAME", scope = 'branch_1x21')

        branch_poola = slim.avg_pool2d(inputs, [1, 5], [1, 1], padding = "SAME", scope = 'branch_poola')
        branch_poolb = slim.conv2d(branch_poola, four, [1, 1], [1, 1], padding = "SAME", scope = 'branch_poolb')
        out = tf.concat(axis = 3, values = [branch_1x3, branch_1x7, branch_1x11, branch_poolb])
        out = slim.utils.collect_named_outputs(outputs_collections, scope.name, out)

        return out
def inceptionv1_arg_scope(weight_decay = 0.0005):
    with slim.arg_scope([slim.variable, slim.model_variable], device = '/cpu:0'):
        with slim.arg_scope(
                [slim.conv2d, slim.fully_connected],
                weights_initializer = slim.variance_scaling_initializer(),
                weights_regularizer = slim.l2_regularizer(weight_decay),
                activation_fn = tf.nn.relu,
                biases_initializer = tf.zeros_initializer()):
            with slim.arg_scope([slim.conv2d], padding = 'SAME'):
                with slim.arg_scope([slim.avg_pool2d], padding = 'SAME') as arg_sc:
                    return arg_sc
                
def inceptionv1(inputs,
          num_classes = 1000,
          dropout_keep_prob = 0.5,
          is_training=True):
    with tf.variable_scope('inceptionv1', [inputs, num_classes]) as sc:
        end_points_collection = sc.original_name_scope + '_end_points'
        # Collect outputs for conv2d, fully_connected and max_pool2d.

        with slim.arg_scope(
                [slim.conv2d, slim.fully_connected, slim.max_pool2d, block],
                outputs_collections = end_points_collection):
            # print(inputs.get_shape()[1].value)
            net = slim.conv2d(inputs, 64, [inputs.get_shape()[1].value, 5], [1, 2],padding="VALID", scope = 'conv1')  # 996
            net = slim.max_pool2d(net, [1, 3], [1, 2], scope = 'pool1')  # 497
            net = slim.conv2d(net, 96, [1, 5], [1, 1], scope = 'conv2')  # 493
            net = slim.max_pool2d(net, [1, 3], [1, 2], scope = 'pool2')  # 246
            net = block(net, 32, 64, 16, 16, 'mixed_128')  # 246
            net = block(net, 64, 96, 48, 32, 'mixed_240')  # 246
            net = slim.max_pool2d(net, [1, 3], [1, 2], scope = 'pool_after_mixed_240')  # 122
            net = block(net, 96, 112, 24, 32, 'mixed_264')  # 122
            net = block(net, 80, 128, 32, 32, 'mixed_272_1')  # 122
            net = slim.max_pool2d(net, [1, 3], [1, 2], scope = 'pool_after_mixed_272_1')  # 60
            net = block(net, 64, 144, 32, 32, 'mixed_272_2')  # 60
            net = block(net, 96, 160, 64, 64, 'mixed_384_1')  # 60
            net = slim.max_pool2d(net, [1, 3], [1, 2], scope = 'pool_after_mixed_384_1')  # 29
            net = block(net, 96, 160, 64, 64, 'mixed_384_2')  # 29
            net = block(net, 128, 192, 80, 80, 'mixed_480')  # 29
            net = slim.max_pool2d(net, [1, 3], [1, 2], scope = 'pool_after_mixed_288')  # 14
            shape = net.get_shape().as_list()
            print(shape)
            net = tf.reshape(net, [-1, shape[1] * shape[2] * shape[3]])
            net = slim.fully_connected(net, 512, scope = "fc1")
            net = slim.fully_connected(net, 512, scope = "fc2")
            # net = slim.conv2d(net, 256, [1, 5], [1, 1], scope = 'conv_end')  # 6
            # print(net)
#             net = slim.avg_pool2d(net, [1, 6], [1, 1],padding="VALID", scope = 'pool_end')  # 1
#             #
#             net = slim.conv2d(net, 512, [1, 1], scope = 'conv_fc1')
#             net = slim.conv2d(net, 512, [1, 1], scope = 'conv_fc2')
           
            # net = slim.conv2d(net, num_classes, [1, 1], activation_fn = None, normalizer_fn = None,
            #                   scope = 'logits')
            net = slim.fully_connected(net, num_classes, activation_fn = None, scope = 'logits')
            if net.get_shape().ndims != 2:
                net = tf.squeeze(net, [1, 2], name = 'SpatialSqueeze_logits')
            end_points = utils.convert_collection_to_dict(end_points_collection)
            if num_classes is not None:
                end_points['predictions'] = slim.softmax(net, scope = 'predictions')
            return net, end_points

    
def _activation_summary(x):

    print("name:", x.op.name, ' ', "shape:", x.get_shape().as_list())

def _activation_summaries(endpoints):
    with tf.name_scope('summaries'):
        print('____________________________model output shape_______________________________')
        for act in endpoints.values():
            _activation_summary(act)
def inference_inceptionv1(images, num_classes, is_training = False):
    with slim.arg_scope(inceptionv1_arg_scope()):
        logits, endpoints = inceptionv1(
            images,
            dropout_keep_prob = 0.5,
            num_classes = num_classes,
            is_training = is_training)

    # Add summaries for viewing model statistics on TensorBoard.
    _activation_summaries(endpoints)

    # Grab the logits associated with the side head. Employed during training.

    return logits

test='test_class1'
model_out='test_class1/model_out'
model_converted='test_class1/model_converted'
if os.path.exists(test):
    t=input("The path(%s) is exist,do you want to delete it and remake?(y/n):\n" %(test,)).strip()
    while t not in ['y','n']:
        t = input("your input is invalid,please input again(y/n):\n")
    if t =='y':
        shutil.rmtree(test)
        os.makedirs(model_out)
        os.makedirs(model_converted)
    else:
        pass
else:
    print("The test path is not exist! make it !\n")
    os.makedirs(model_out)
    os.makedirs(model_converted)
       
    
with tf.Graph().as_default() as g:
    images=tf.placeholder(dtype=tf.float32,shape=(None,3,1000,1),name='input_node')
    logits = inference_inceptionv1(images, 12, is_training = False)
    probs = tf.nn.softmax(logits,name='myoutputnode')
    saver=tf.train.Saver()
    tf.set_random_seed(123)
    init = tf.global_variables_initializer()
    with tf.Session() as sess:
        sess.run(init)
        saver.save(sess,model_out+'/model.ckpt')
        graph = tf.get_default_graph()
        input_graph_def = graph.as_graph_def()
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            input_graph_def,
            ['myoutputnode']
        )
        outgraph=tf.graph_util.remove_training_nodes(output_graph_def)
        for node in outgraph.node:
            print('node:',node.name)
        tf.train.write_graph(outgraph,model_out,'frozen_model.pb',as_text=False)
        tf.train.write_graph(outgraph,model_out,'frozen_model.pbtxt')
            

The test path is not exist! make it !

[None, 1, 6, 480]
____________________________model output shape_______________________________
name: inceptionv1/conv1/Relu   shape: [None, 1, 498, 64]
name: inceptionv1/pool1/MaxPool   shape: [None, 1, 248, 64]
name: inceptionv1/conv2/Relu   shape: [None, 1, 248, 96]
name: inceptionv1/pool2/MaxPool   shape: [None, 1, 123, 96]
name: inceptionv1/mixed_128/branch_1x5/Relu   shape: [None, 1, 123, 32]
name: inceptionv1/mixed_128/branch_1x9/Relu   shape: [None, 1, 123, 64]
name: inceptionv1/mixed_128/branch_1x21/Relu   shape: [None, 1, 123, 16]
name: inceptionv1/mixed_128/branch_poolb/Relu   shape: [None, 1, 123, 16]
name: inceptionv1/mixed_128/concat   shape: [None, 1, 123, 128]
name: inceptionv1/mixed_240/branch_1x5/Relu   shape: [None, 1, 123, 64]
name: inceptionv1/mixed_240/branch_1x9/Relu   shape: [None, 1, 123, 96]
name: inceptionv1/mixed_240/branch_1x21/Relu   shape: [None, 1, 123, 48]
name: inceptionv1/mixed_240/branch_poolb/Relu   shape: [Non

In [9]:
%%bash 
scribe="/opt/intel/openvino/deployment_tools/model_optimizer/mo_tf.py"
python3 $scribe --input input_node --output myoutputnode --input_model ./test_class1/model_out/frozen_model.pb --model_name model_fp16 --output_dir ./test_class1/model_converted/model_fp16 --log_level=ERROR --data_type=FP16 --input_shape [1,3,1000,1]

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/tianliang/2019r3/./test_class1/model_out/frozen_model.pb
	- Path for generated IR: 	/home/tianliang/2019r3/./test_class1/model_converted/model_fp16
	- IR output name: 	model_fp16
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	input_node
	- Output layers: 	myoutputnode
	- Input shapes: 	[1,3,1000,1]
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP16
	- Enable fusing: 	True
	- Enable grouped convolutions fusing: 	True
	- Move mean values to preprocess section: 	False
	- Reverse input channels: 	False
TensorFlow specific parameters:
	- Input model in text protobuf format: 	False
	- Path to model dump for TensorBoard: 	None
	- List of shared libraries with TensorFlow custom layers implementation: 	None
	- Update the configuration file with input/output node names: 	None
	- Use configuration file use

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [9]:
data1 = np.random.rand(72,500).flatten()

# 使用pb调用

In [18]:
import tensorflow as tf
import numpy as np


classid_str = "classid"
probability_str = "probability"
n,h,w,c=1,1,500,1
labels_map=None

def load_graph(file_path):

    with tf.gfile.GFile(file_path,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        with tf.Graph().as_default() as graph:
            tf.import_graph_def(graph_def,input_map = None,return_elements = None,name = "",op_dict = None,producer_op_list = None)
            graph_nodes = [n for n in graph_def.node]
            return graph,graph_nodes
pb_path = "test/model_out/frozen_model.pb"
graph,graph_nodes = load_graph(pb_path)
# for node in graph_nodes:
#     print('node:', node.name)
input = graph.get_tensor_by_name('input_node:0')
output = graph.get_tensor_by_name('myoutputnode:0')


data=np.load('data1.npy').reshape(1,h,w,c)
with tf.Session(graph=graph) as sess:
    softmax_values = sess.run(output, feed_dict = {input: data})
    
    for i, probs in enumerate(softmax_values):
        probs = np.squeeze(probs)
        print(probs)
#         top_ind = np.argsort(probs)[-:][::-1]
#     print(classid_str, probability_str)
#     print("{} {}".format('-' * len(classid_str), '-' * len(probability_str)))
#     for id in top_ind:
#         det_label = labels_map[id] if labels_map else "{}".format(id)
#         label_length = len(det_label)
#         space_num_before = (len(classid_str) - label_length) // 2
#         space_num_after = len(classid_str) - (space_num_before + label_length) + 2
#         space_num_before_prob = (len(probability_str) - len(str(probs[id]))) // 2
#         print("{}{}{}{}{:.7f}".format(' ' * space_num_before, det_label,
#                                       ' ' * space_num_after, ' ' * space_num_before_prob,
#                                       probs[id]))    

2020-01-07 19:22:27,353 WARNING:From <ipython-input-18-690f10e4cff2>:16: calling import_graph_def (from tensorflow.python.framework.importer) with op_dict is deprecated and will be removed in a future version.
Instructions for updating:
Please file an issue at https://github.com/tensorflow/tensorflow/issues if you depend on this feature.
[0.2996923  0.04790032 0.6524074 ]


# CPU fp16

In [19]:
import sys
import os
import numpy as np
from time import time
from openvino.inference_engine import IENetwork, IEPlugin


class ObjDict(dict):
    """Makes a dictionary behave like an object, with attribute-style access.
    """

    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)

    def __setattr__(self, name, value):
        self[name] = value
args=ObjDict()

args.device="CPU"
args.plugin_dir=None
if args.device in ['CPU']:
    args.model="test/model_converted/model_fp16/model_fp16.xml"
elif args.device in ['MYRIAD']:
    args.model="test/model_converted/model_fp16/model_fp16.xml"
else:
    args.model="test/model_converted/model_fp32/model_fp32.xml"
    #args.model="/home/ubuntu/tf_models/test20190704/model_converted/model_fp16/model_fp16.xml"

args.number_iter=100
args.perf_counts=True
args.number_top=5
args.labels=None

plugin = IEPlugin(device=args.device, plugin_dirs=args.plugin_dir)
model_xml=args.model
model_bin = os.path.splitext(model_xml)[0] + ".bin"
net = IENetwork(model=model_xml, weights=model_bin)

if plugin.device == "CPU":
    supported_layers = plugin.get_supported_layers(net)
    not_supported_layers = [l for l in net.layers.keys() if l not in supported_layers]
    if len(not_supported_layers) != 0:
        log.error("Following layers are not supported by the plugin for specified device {}:\n {}".
                  format(plugin.device, ', '.join(not_supported_layers)))
        log.error("Please try to specify cpu extensions library path in sample's command line parameters using -l "
                  "or --cpu_extension command line argument")
        sys.exit(1)
        
log.info("Preparing input blobs")
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
#net.batch_size = len(args.input)
n,c,h,w=net.inputs[input_blob].shape
print(n,c,h,w)

for i in range(n):
    np.random.seed(5)

    args.input=np.load('data1.npy')
    images = args.input.transpose((0,3, 1, 2))  # Change data layout from HWC to CHW

log.info("Batch size is {}".format(n))
# Loading model to the plugin
log.info("Loading model to the plugin")
exec_net = plugin.load(network=net)
# Start sync inference
log.info("Starting inference ({} iterations)".format(args.number_iter))
infer_time = []
for i in range(args.number_iter):
    t0 = time()
    res = exec_net.infer(inputs={input_blob: images})
    infer_time.append((time() - t0) * 1000)
log.info("Average running time of one iteration: {} ms".format(np.average(np.asarray(infer_time))))
log.info("Performance {} FPS".format(int(1000/np.average(np.asarray(infer_time)))))

log.info("Processing output blob")
res = res[out_blob]
log.info("Top {} results: ".format(args.number_top))
if args.labels:
    with open(args.labels, 'r') as f:
        labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip() for x in f]
else:
    labels_map = None
    
classid_str = "classid"
probability_str = "probability"
for i, probs in enumerate(res):
    probs = np.squeeze(probs)
    top_ind = np.argsort(probs)[-args.number_top:][::-1]
    #print("Image {}\n".format(args.input[i]))
    print(classid_str, probability_str)
    print("{} {}".format('-' * len(classid_str), '-' * len(probability_str)))
    for id in top_ind:
        det_label = labels_map[id] if labels_map else "{}".format(id)
        label_length = len(det_label)
        space_num_before = (len(classid_str) - label_length) // 2
        space_num_after = len(classid_str) - (space_num_before + label_length) + 2
        space_num_before_prob = (len(probability_str) - len(str(probs[id]))) // 2
        print("{}{}{}{}{:.7f}".format(' ' * space_num_before, det_label,
                                      ' ' * space_num_after, ' ' * space_num_before_prob,
                                      probs[id]))    
if args.perf_counts:
    perf_counts = exec_net.requests[0].get_perf_counts()
    log.info("Performance counters:")
    print("{:<60} {:<15} {:<25} {:<15} {:<10}".format('name', 'layer_type', 'exet_type', 'status', 'real_time, us'))
    for layer, stats in perf_counts.items():
        layer = layer.split('/')[-1]
        print("{:<60} {:<15} {:<25} {:<15} {:<10}".format(layer, stats['layer_type'], stats['exec_type'],
                                                          stats['status'], stats['real_time']))

2020-01-07 19:22:32,720 INFO:Preparing input blobs
1 1 1 500
2020-01-07 19:22:32,722 INFO:Batch size is 1
2020-01-07 19:22:32,722 INFO:Loading model to the plugin
2020-01-07 19:22:33,183 INFO:Starting inference (100 iterations)
2020-01-07 19:22:33,341 INFO:Average running time of one iteration: 1.5633392333984375 ms
2020-01-07 19:22:33,342 INFO:Performance 639 FPS
2020-01-07 19:22:33,343 INFO:Processing output blob
2020-01-07 19:22:33,343 INFO:Top 5 results: 
classid probability
------- -----------
   2     0.6525519
   0     0.2995729
   1     0.0478752
2020-01-07 19:22:33,344 INFO:Performance counters:
name                                                         layer_type      exet_type                 status          real_time, us
Relu                                                         ReLU            undef                     NOT_RUN         0         
MatMul                                                       FullyConnected  gemm_blas_FP32            EXECUTED        29    

In [18]:
np.save('test_data.npy',data1)

# 以下为测试

# MYRIAD  fp16

In [20]:
import sys
import os
import numpy as np
from time import time
from openvino.inference_engine import IENetwork, IEPlugin


class ObjDict(dict):
    """Makes a dictionary behave like an object, with attribute-style access.
    """

    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)

    def __setattr__(self, name, value):
        self[name] = value
args=ObjDict()

args.device="MYRIAD"
args.plugin_dir=None
if args.device in ['CPU']:
    args.model="test/model_converted/model_fp16/model_fp16.xml"
elif args.device in ['MYRIAD']:
    args.model="test/model_converted/model_fp16/model_fp16.xml"
else:
    args.model="test/model_converted/model_fp32/model_fp32.xml"
    #args.model="/home/ubuntu/tf_models/test20190704/model_converted/model_fp16/model_fp16.xml"

args.number_iter=100
args.perf_counts=True
args.number_top=5
args.labels=None

plugin = IEPlugin(device=args.device, plugin_dirs=args.plugin_dir)
model_xml=args.model
model_bin = os.path.splitext(model_xml)[0] + ".bin"
net = IENetwork(model=model_xml, weights=model_bin)

if plugin.device == "CPU":
    supported_layers = plugin.get_supported_layers(net)
    not_supported_layers = [l for l in net.layers.keys() if l not in supported_layers]
    if len(not_supported_layers) != 0:
        log.error("Following layers are not supported by the plugin for specified device {}:\n {}".
                  format(plugin.device, ', '.join(not_supported_layers)))
        log.error("Please try to specify cpu extensions library path in sample's command line parameters using -l "
                  "or --cpu_extension command line argument")
        sys.exit(1)
        
log.info("Preparing input blobs")
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
#net.batch_size = len(args.input)
n,c,h,w=net.inputs[input_blob].shape
print(n,c,h,w)

for i in range(n):
    np.random.seed(5)
    args.input=np.load('data1.npy')
    images = args.input.transpose((0,3, 1, 2))  # Change data layout from HWC to CHW

log.info("Batch size is {}".format(n))
# Loading model to the plugin
log.info("Loading model to the plugin")
exec_net = plugin.load(network=net)
# Start sync inference
log.info("Starting inference ({} iterations)".format(args.number_iter))
infer_time = []
for i in range(args.number_iter):
    t0 = time()
    res = exec_net.infer(inputs={input_blob: images})
    infer_time.append((time() - t0) * 1000)
log.info("Average running time of one iteration: {} ms".format(np.average(np.asarray(infer_time))))
log.info("Performance {:.2f} FPS".format((1000/np.average(np.asarray(infer_time)))))

log.info("Processing output blob")
res = res[out_blob]
log.info("Top {} results: ".format(args.number_top))
if args.labels:
    with open(args.labels, 'r') as f:
        labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip() for x in f]
else:
    labels_map = None
    
classid_str = "classid"
probability_str = "probability"
for i, probs in enumerate(res):
    probs = np.squeeze(probs)
    top_ind = np.argsort(probs)[-args.number_top:][::-1]
    #print("Image {}\n".format(args.input[i]))
    print(classid_str, probability_str)
    print("{} {}".format('-' * len(classid_str), '-' * len(probability_str)))
    for id in top_ind:
        det_label = labels_map[id] if labels_map else "{}".format(id)
        label_length = len(det_label)
        space_num_before = (len(classid_str) - label_length) // 2
        space_num_after = len(classid_str) - (space_num_before + label_length) + 2
        space_num_before_prob = (len(probability_str) - len(str(probs[id]))) // 2
        print("{}{}{}{}{:.7f}".format(' ' * space_num_before, det_label,
                                      ' ' * space_num_after, ' ' * space_num_before_prob,
                                      probs[id]))    
if args.perf_counts:
    perf_counts = exec_net.requests[0].get_perf_counts()
    log.info("Performance counters:")
    print("{:<60} {:<15} {:<25} {:<15} {:<10}".format('name', 'layer_type', 'exet_type', 'status', 'real_time, us'))
    for layer, stats in perf_counts.items():
        layer = layer.split('/')[-1]
        print("{:<60} {:<15} {:<25} {:<15} {:<10}".format(layer, stats['layer_type'], stats['exec_type'],
                                                          stats['status'], stats['real_time']))

2020-01-07 19:22:37,857 INFO:Preparing input blobs
1 1 1 500
2020-01-07 19:22:37,859 INFO:Batch size is 1
2020-01-07 19:22:37,860 INFO:Loading model to the plugin
2020-01-07 19:22:39,865 INFO:Starting inference (100 iterations)
2020-01-07 19:22:40,355 INFO:Average running time of one iteration: 4.889945983886719 ms
2020-01-07 19:22:40,356 INFO:Performance 204.50 FPS
2020-01-07 19:22:40,356 INFO:Processing output blob
2020-01-07 19:22:40,357 INFO:Top 5 results: 
classid probability
------- -----------
   2      0.6484375
   0     0.3024902
   1     0.0493164
2020-01-07 19:22:40,359 INFO:Performance counters:
name                                                         layer_type      exet_type                 status          real_time, us
Relu                                                         ReLU            <none>                    OPTIMIZED_OUT   0         
MatMul                                                       FullyConnected  MyriadXHwOp               EXECUTED        53 

# 第二个分类模型

In [26]:
# pb

In [22]:
import tensorflow as tf
import numpy as np


classid_str = "classid"
probability_str = "probability"
n,h,w,c=1,3,1000,1
labels_map=None

def load_graph(file_path):

    with tf.gfile.GFile(file_path,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        with tf.Graph().as_default() as graph:
            tf.import_graph_def(graph_def,input_map = None,return_elements = None,name = "",op_dict = None,producer_op_list = None)
            graph_nodes = [n for n in graph_def.node]
            return graph,graph_nodes
pb_path = "test_class/model_out/frozen_model.pb"
graph,graph_nodes = load_graph(pb_path)
# for node in graph_nodes:
#     print('node:', node.name)
input = graph.get_tensor_by_name('input_node:0')
output = graph.get_tensor_by_name('myoutputnode:0')


data=np.load('data2.npy').reshape(1,h,w,c)
with tf.Session(graph=graph) as sess:
    softmax_values = sess.run(output, feed_dict = {input: data})
    
    for i, probs in enumerate(softmax_values):
        probs = np.squeeze(probs)
        print(probs)
#         top_ind = np.argsort(probs)[-:][::-1]
#     print(classid_str, probability_str)
#     print("{} {}".format('-' * len(classid_str), '-' * len(probability_str)))
#     for id in top_ind:
#         det_label = labels_map[id] if labels_map else "{}".format(id)
#         label_length = len(det_label)
#         space_num_before = (len(classid_str) - label_length) // 2
#         space_num_after = len(classid_str) - (space_num_before + label_length) + 2
#         space_num_before_prob = (len(probability_str) - len(str(probs[id]))) // 2
#         print("{}{}{}{}{:.7f}".format(' ' * space_num_before, det_label,
#                                       ' ' * space_num_after, ' ' * space_num_before_prob,
#                                       probs[id]))    

[3.2618225e-03 6.2778038e-03 1.0355789e-02 9.5154536e-01 1.8674359e-04
 5.0049443e-03 4.7170813e-03 3.7591392e-03 9.4591480e-05 5.0033408e-04
 1.4174117e-02 1.2224575e-04]


In [2]:
import sys
import os
import numpy as np
from time import time
from openvino.inference_engine import IENetwork, IEPlugin


class ObjDict(dict):
    """Makes a dictionary behave like an object, with attribute-style access.
    """

    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)

    def __setattr__(self, name, value):
        self[name] = value
args=ObjDict()

args.device="CPU"
args.plugin_dir=None
if args.device in ['CPU']:
    args.model="test_class/model_converted/model_fp16/model_fp16.xml"
elif args.device in ['MYRIAD']:
    args.model="test_class/model_converted/model_fp16/model_fp16.xml"
else:
    args.model="test_class/model_converted/model_fp32/model_fp32.xml"
    #args.model="/home/ubuntu/tf_models/test20190704/model_converted/model_fp16/model_fp16.xml"

args.number_iter=100
args.perf_counts=True
args.number_top=13
args.labels=None

plugin = IEPlugin(device=args.device, plugin_dirs=args.plugin_dir)
model_xml=args.model
model_bin = os.path.splitext(model_xml)[0] + ".bin"
net = IENetwork(model=model_xml, weights=model_bin)

if plugin.device == "CPU":
    supported_layers = plugin.get_supported_layers(net)
    not_supported_layers = [l for l in net.layers.keys() if l not in supported_layers]
    if len(not_supported_layers) != 0:
        log.error("Following layers are not supported by the plugin for specified device {}:\n {}".
                  format(plugin.device, ', '.join(not_supported_layers)))
        log.error("Please try to specify cpu extensions library path in sample's command line parameters using -l "
                  "or --cpu_extension command line argument")
        sys.exit(1)
        
log.info("Preparing input blobs")
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
#net.batch_size = len(args.input)
n,c,h,w=net.inputs[input_blob].shape
print(n,c,h,w)

for i in range(n):
    np.random.seed(5)

    args.input=np.load('data2.npy')
    images = args.input.transpose((0,3, 1, 2))  # Change data layout from HWC to CHW

log.info("Batch size is {}".format(n))
# Loading model to the plugin
log.info("Loading model to the plugin")
exec_net = plugin.load(network=net)
# Start sync inference
log.info("Starting inference ({} iterations)".format(args.number_iter))
infer_time = []
for i in range(args.number_iter):
    t0 = time()
    res = exec_net.infer(inputs={input_blob: images})
    infer_time.append((time() - t0) * 1000)
log.info("Average running time of one iteration: {} ms".format(np.average(np.asarray(infer_time))))
log.info("Performance {} FPS".format(int(1000/np.average(np.asarray(infer_time)))))

log.info("Processing output blob")
res = res[out_blob]
log.info("Top {} results: ".format(args.number_top))
if args.labels:
    with open(args.labels, 'r') as f:
        labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip() for x in f]
else:
    labels_map = None
    
classid_str = "classid"
probability_str = "probability"
for i, probs in enumerate(res):
    probs = np.squeeze(probs)
    top_ind = np.argsort(probs)[-args.number_top:][::-1]
    #print("Image {}\n".format(args.input[i]))
    print(classid_str, probability_str)
    print("{} {}".format('-' * len(classid_str), '-' * len(probability_str)))
    for id in top_ind:
        det_label = labels_map[id] if labels_map else "{}".format(id)
        label_length = len(det_label)
        space_num_before = (len(classid_str) - label_length) // 2
        space_num_after = len(classid_str) - (space_num_before + label_length) + 2
        space_num_before_prob = (len(probability_str) - len(str(probs[id]))) // 2
        print("{}{}{}{}{:.7f}".format(' ' * space_num_before, det_label,
                                      ' ' * space_num_after, ' ' * space_num_before_prob,
                                      probs[id]))    
if args.perf_counts:
    perf_counts = exec_net.requests[0].get_perf_counts()
    log.info("Performance counters:")
    print("{:<60} {:<15} {:<25} {:<15} {:<10}".format('name', 'layer_type', 'exet_type', 'status', 'real_time, us'))
    for layer, stats in perf_counts.items():
        layer = layer.split('/')[-1]
        print("{:<60} {:<15} {:<25} {:<15} {:<10}".format(layer, stats['layer_type'], stats['exec_type'],
                                                          stats['status'], stats['real_time']))

2020-01-07 19:24:22,983 ERROR:Following layers are not supported by the plugin for specified device CPU:
 inceptionv1/SpatialSqueeze_logits
2020-01-07 19:24:22,984 ERROR:Please try to specify cpu extensions library path in sample's command line parameters using -l or --cpu_extension command line argument


SystemExit: 1

/usr/local/lib/python3.5/dist-packages/IPython/core/interactiveshell.py:3334: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [3]:
import sys
import os
import numpy as np
from time import time
from openvino.inference_engine import IENetwork, IEPlugin


class ObjDict(dict):
    """Makes a dictionary behave like an object, with attribute-style access.
    """

    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)

    def __setattr__(self, name, value):
        self[name] = value
args=ObjDict()

args.device="MYRIAD"
args.plugin_dir=None
if args.device in ['CPU']:
    args.model="test_class/model_converted/model_fp16/model_fp16.xml"
elif args.device in ['MYRIAD']:
    args.model="test_class/model_converted/model_fp16/model_fp16.xml"
else:
    args.model="test_class/model_converted/model_fp32/model_fp32.xml"
    #args.model="/home/ubuntu/tf_models/test20190704/model_converted/model_fp16/model_fp16.xml"

args.number_iter=100
args.perf_counts=True
args.number_top=13
args.labels=None

plugin = IEPlugin(device=args.device, plugin_dirs=args.plugin_dir)
model_xml=args.model
model_bin = os.path.splitext(model_xml)[0] + ".bin"
net = IENetwork(model=model_xml, weights=model_bin)

if plugin.device == "CPU":
    supported_layers = plugin.get_supported_layers(net)
    not_supported_layers = [l for l in net.layers.keys() if l not in supported_layers]
    if len(not_supported_layers) != 0:
        log.error("Following layers are not supported by the plugin for specified device {}:\n {}".
                  format(plugin.device, ', '.join(not_supported_layers)))
        log.error("Please try to specify cpu extensions library path in sample's command line parameters using -l "
                  "or --cpu_extension command line argument")
        sys.exit(1)
        
log.info("Preparing input blobs")
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
#net.batch_size = len(args.input)
n,c,h,w=net.inputs[input_blob].shape
print(n,c,h,w)

for i in range(n):
    np.random.seed(5)

    args.input=np.load('data2.npy')
    images = args.input.transpose((0,3, 1, 2))  # Change data layout from HWC to CHW

log.info("Batch size is {}".format(n))
# Loading model to the plugin
log.info("Loading model to the plugin")
exec_net = plugin.load(network=net)
# Start sync inference
log.info("Starting inference ({} iterations)".format(args.number_iter))
infer_time = []
for i in range(args.number_iter):
    t0 = time()
    res = exec_net.infer(inputs={input_blob: images})
    infer_time.append((time() - t0) * 1000)
log.info("Average running time of one iteration: {} ms".format(np.average(np.asarray(infer_time))))
log.info("Performance {:.2f} FPS".format((1000/np.average(np.asarray(infer_time)))))

log.info("Processing output blob")
res = res[out_blob]
log.info("Top {} results: ".format(args.number_top))
if args.labels:
    with open(args.labels, 'r') as f:
        labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip() for x in f]
else:
    labels_map = None
    
classid_str = "classid"
probability_str = "probability"
for i, probs in enumerate(res):
    probs = np.squeeze(probs)
    top_ind = np.argsort(probs)[-args.number_top:][::-1]
    #print("Image {}\n".format(args.input[i]))
    print(classid_str, probability_str)
    print("{} {}".format('-' * len(classid_str), '-' * len(probability_str)))
    for id in top_ind:
        det_label = labels_map[id] if labels_map else "{}".format(id)
        label_length = len(det_label)
        space_num_before = (len(classid_str) - label_length) // 2
        space_num_after = len(classid_str) - (space_num_before + label_length) + 2
        space_num_before_prob = (len(probability_str) - len(str(probs[id]))) // 2
        print("{}{}{}{}{:.7f}".format(' ' * space_num_before, det_label,
                                      ' ' * space_num_after, ' ' * space_num_before_prob,
                                      probs[id]))    
if args.perf_counts:
    perf_counts = exec_net.requests[0].get_perf_counts()
    log.info("Performance counters:")
    print("{:<60} {:<15} {:<25} {:<15} {:<10}".format('name', 'layer_type', 'exet_type', 'status', 'real_time, us'))
    for layer, stats in perf_counts.items():
        layer = layer.split('/')[-1]
        print("{:<60} {:<15} {:<25} {:<15} {:<10}".format(layer, stats['layer_type'], stats['exec_type'],
                                                          stats['status'], stats['real_time']))

2020-01-07 19:24:37,011 INFO:Preparing input blobs
1 1 3 1000
2020-01-07 19:24:37,014 INFO:Batch size is 1
2020-01-07 19:24:37,014 INFO:Loading model to the plugin
2020-01-07 19:24:39,118 INFO:Starting inference (100 iterations)
2020-01-07 19:24:41,553 INFO:Average running time of one iteration: 24.34601068496704 ms
2020-01-07 19:24:41,554 INFO:Performance 41.07 FPS
2020-01-07 19:24:41,554 INFO:Processing output blob
2020-01-07 19:24:41,555 INFO:Top 13 results: 
classid probability
------- -----------
   3      0.9428711
  10     0.0159912
   2     0.0113297
   1     0.0066643
   5     0.0064659
   6     0.0064163
   7     0.0050583
   0     0.0040703
   9     0.0006032
   4     0.0002429
  11     0.0001874
   8     0.0001196
2020-01-07 19:24:41,557 INFO:Performance counters:
name                                                         layer_type      exet_type                 status          real_time, us
MaxPool                                                      Pooling         Max

In [2]:
import sys
import os
import numpy as np
from time import time
from openvino.inference_engine import IENetwork, IEPlugin


class ObjDict(dict):
    """Makes a dictionary behave like an object, with attribute-style access.
    """

    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)

    def __setattr__(self, name, value):
        self[name] = value
args=ObjDict()

args.device="MYRIAD"
args.plugin_dir=None
if args.device in ['CPU']:
    args.model="test_class1/model_converted/model_fp16/model_fp16.xml"
elif args.device in ['MYRIAD']:
    args.model="test_class1/model_converted/model_fp16/model_fp16.xml"
else:
    args.model="test_class1/model_converted/model_fp32/model_fp32.xml"
    #args.model="/home/ubuntu/tf_models/test20190704/model_converted/model_fp16/model_fp16.xml"

args.number_iter=100
args.perf_counts=True
args.number_top=13
args.labels=None

plugin = IEPlugin(device=args.device, plugin_dirs=args.plugin_dir)
model_xml=args.model
model_bin = os.path.splitext(model_xml)[0] + ".bin"
net = IENetwork(model=model_xml, weights=model_bin)

if plugin.device == "CPU":
    supported_layers = plugin.get_supported_layers(net)
    not_supported_layers = [l for l in net.layers.keys() if l not in supported_layers]
    if len(not_supported_layers) != 0:
        log.error("Following layers are not supported by the plugin for specified device {}:\n {}".
                  format(plugin.device, ', '.join(not_supported_layers)))
        log.error("Please try to specify cpu extensions library path in sample's command line parameters using -l "
                  "or --cpu_extension command line argument")
        sys.exit(1)
        
log.info("Preparing input blobs")
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
#net.batch_size = len(args.input)
n,c,h,w=net.inputs[input_blob].shape
print(n,c,h,w)

for i in range(n):
    np.random.seed(5)

    args.input=np.load('data2.npy')
    images = args.input.transpose((0,3, 1, 2))  # Change data layout from HWC to CHW

log.info("Batch size is {}".format(n))
# Loading model to the plugin
log.info("Loading model to the plugin")
exec_net = plugin.load(network=net)
# Start sync inference
log.info("Starting inference ({} iterations)".format(args.number_iter))
infer_time = []
for i in range(args.number_iter):
    t0 = time()
    res = exec_net.infer(inputs={input_blob: images})
    infer_time.append((time() - t0) * 1000)
log.info("Average running time of one iteration: {} ms".format(np.average(np.asarray(infer_time))))
log.info("Performance {:.2f} FPS".format((1000/np.average(np.asarray(infer_time)))))

log.info("Processing output blob")
res = res[out_blob]
log.info("Top {} results: ".format(args.number_top))
if args.labels:
    with open(args.labels, 'r') as f:
        labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip() for x in f]
else:
    labels_map = None
    
classid_str = "classid"
probability_str = "probability"
for i, probs in enumerate(res):
    probs = np.squeeze(probs)
    top_ind = np.argsort(probs)[-args.number_top:][::-1]
    #print("Image {}\n".format(args.input[i]))
    print(classid_str, probability_str)
    print("{} {}".format('-' * len(classid_str), '-' * len(probability_str)))
    for id in top_ind:
        det_label = labels_map[id] if labels_map else "{}".format(id)
        label_length = len(det_label)
        space_num_before = (len(classid_str) - label_length) // 2
        space_num_after = len(classid_str) - (space_num_before + label_length) + 2
        space_num_before_prob = (len(probability_str) - len(str(probs[id]))) // 2
        print("{}{}{}{}{:.7f}".format(' ' * space_num_before, det_label,
                                      ' ' * space_num_after, ' ' * space_num_before_prob,
                                      probs[id]))    
if args.perf_counts:
    perf_counts = exec_net.requests[0].get_perf_counts()
    log.info("Performance counters:")
    print("{:<60} {:<15} {:<25} {:<15} {:<10}".format('name', 'layer_type', 'exet_type', 'status', 'real_time, us'))
    for layer, stats in perf_counts.items():
        layer = layer.split('/')[-1]
        print("{:<60} {:<15} {:<25} {:<15} {:<10}".format(layer, stats['layer_type'], stats['exec_type'],
                                                          stats['status'], stats['real_time']))

2020-01-07 20:07:13,149 INFO:Preparing input blobs
1 1 3 1000
2020-01-07 20:07:13,150 INFO:Batch size is 1
2020-01-07 20:07:13,151 INFO:Loading model to the plugin
2020-01-07 20:07:15,325 INFO:Starting inference (100 iterations)
2020-01-07 20:07:17,795 INFO:Average running time of one iteration: 24.68571901321411 ms
2020-01-07 20:07:17,796 INFO:Performance 40.51 FPS
2020-01-07 20:07:17,797 INFO:Processing output blob
2020-01-07 20:07:17,797 INFO:Top 13 results: 
classid probability
------- -----------
   6      0.6801758
  10     0.0928345
   7     0.0728760
   3     0.0479431
   1      0.0340271
   8     0.0271301
  11     0.0204620
   9     0.0099869
   4     0.0096436
   2     0.0038815
   5     0.0007901
   0     0.0003078
2020-01-07 20:07:17,801 INFO:Performance counters:
name                                                         layer_type      exet_type                 status          real_time, us
concat                                                       Concat          Co

# 2020年1月13日最新测试

## model1 同步

In [1]:
import sys
import os
import numpy as np
from time import time
import logging as log
#reload(logging)
from openvino.inference_engine import IENetwork, IECore
log.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=log.INFO,stream=sys.stdout)

class ObjDict(dict):
    """Makes a dictionary behave like an object, with attribute-style access.
    """

    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)

    def __setattr__(self, name, value):
        self[name] = value
args=ObjDict()
args.device="MYRIAD"
args.number_iter=1000
args.perf_counts=True
args.number_top=5
args.labels=None


if args.device in ['CPU']:
    args.model="test/model_converted/model_fp16/model_fp16.xml"
elif args.device in ['MYRIAD']:
    args.model="test/model_converted/model_fp16/model_fp16.xml"
else:
    args.model="test/model_converted/model_fp32/model_fp32.xml"
    #args.model="/home/ubuntu/tf_models/test20190704/model_converted/model_fp16/model_fp16.xml"
    
model_xml=args.model
model_bin = os.path.splitext(model_xml)[0] + ".bin"

log.info("Creating Inference Engine")
ie = IECore()
log.info("Loading network files:\n\t{}\n\t{}".format(model_xml, model_bin))
net = IENetwork(model=model_xml, weights=model_bin)

log.info("Preparing input blobs")
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
n,c,h,w=net.inputs[input_blob].shape
print(n,c,h,w)

for i in range(n):
    np.random.seed(5)
    args.input=np.load('data1.npy')
    images = args.input.transpose((0,3, 1, 2))  # Change data layout from HWC to CHW

log.info("Batch size is {}".format(n))
# Loading model to the plugin
log.info("Loading model to the plugin")
exec_net = ie.load_network(network=net, device_name=args.device)
log.info("Starting inference in synchronous mode")
log.info("Starting inference ({} iterations)".format(args.number_iter))
infer_time = []
for i in range(args.number_iter):
    t0 = time()
    res = exec_net.infer(inputs={input_blob: images})
    infer_time.append((time() - t0) * 1000)
log.info("Average running time of one iteration: {} ms".format(np.average(np.asarray(infer_time))))
log.info("Performance {:.2f} FPS".format((1000/np.average(np.asarray(infer_time)))))

log.info("Processing output blob")
res = res[out_blob]
log.info("Top {} results: ".format(args.number_top))
if args.labels:
    with open(args.labels, 'r') as f:
        labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip() for x in f]
else:
    labels_map = None
    
classid_str = "classid"
probability_str = "probability"
for i, probs in enumerate(res):
    probs = np.squeeze(probs)
    top_ind = np.argsort(probs)[-args.number_top:][::-1]
    #print("Image {}\n".format(args.input[i]))
    print(classid_str, probability_str)
    print("{} {}".format('-' * len(classid_str), '-' * len(probability_str)))
    for id in top_ind:
        det_label = labels_map[id] if labels_map else "{}".format(id)
        label_length = len(det_label)
        space_num_before = (len(classid_str) - label_length) // 2
        space_num_after = len(classid_str) - (space_num_before + label_length) + 2
        space_num_before_prob = (len(probability_str) - len(str(probs[id]))) // 2
        print("{}{}{}{}{:.7f}".format(' ' * space_num_before, det_label,
                                      ' ' * space_num_after, ' ' * space_num_before_prob,
                                      probs[id]))    
if args.perf_counts:
    perf_counts = exec_net.requests[0].get_perf_counts()
    log.info("Performance counters:")
    print("{:<60} {:<15} {:<25} {:<15} {:<10}".format('name', 'layer_type', 'exet_type', 'status', 'real_time, us'))
    for layer, stats in perf_counts.items():
        layer = layer.split('/')[-1]
        print("{:<60} {:<15} {:<25} {:<15} {:<10}".format(layer, stats['layer_type'], stats['exec_type'],
                                                          stats['status'], stats['real_time']))

2020-01-13 11:21:35,483 INFO:Creating Inference Engine
2020-01-13 11:21:35,484 INFO:Loading network files:
	test/model_converted/model_fp16/model_fp16.xml
	test/model_converted/model_fp16/model_fp16.bin
2020-01-13 11:21:35,488 INFO:Preparing input blobs
1 1 1 500
2020-01-13 11:21:35,490 INFO:Batch size is 1
2020-01-13 11:21:35,491 INFO:Loading model to the plugin
2020-01-13 11:21:37,496 INFO:Starting inference in synchronous mode
2020-01-13 11:21:37,497 INFO:Starting inference (1000 iterations)
2020-01-13 11:21:42,314 INFO:Average running time of one iteration: 4.815056085586548 ms
2020-01-13 11:21:42,314 INFO:Performance 207.68 FPS
2020-01-13 11:21:42,315 INFO:Processing output blob
2020-01-13 11:21:42,315 INFO:Top 5 results: 
classid probability
------- -----------
   2      0.6484375
   0     0.3024902
   1     0.0493164
2020-01-13 11:21:42,317 INFO:Performance counters:
name                                                         layer_type      exet_type                 status    

## model2 同步

In [1]:
import sys
import os
import numpy as np
from time import time
import logging as log
#reload(logging)
from openvino.inference_engine import IENetwork, IECore
log.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=log.INFO,stream=sys.stdout)

class ObjDict(dict):
    """Makes a dictionary behave like an object, with attribute-style access.
    """

    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)

    def __setattr__(self, name, value):
        self[name] = value
args=ObjDict()
args.device="MYRIAD"
args.number_iter=1000
args.perf_counts=True
args.number_top=13
args.labels=None


if args.device in ['CPU']:
    args.model="test_class/model_converted/model_fp16/model_fp16.xml"
elif args.device in ['MYRIAD']:
    args.model="test_class/model_converted/model_fp16/model_fp16.xml"
else:
    args.model="test_class/model_converted/model_fp32/model_fp32.xml"
    #args.model="/home/ubuntu/tf_models/test20190704/model_converted/model_fp16/model_fp16.xml"
    
model_xml=args.model
model_bin = os.path.splitext(model_xml)[0] + ".bin"

log.info("Creating Inference Engine")
ie = IECore()
log.info("Loading network files:\n\t{}\n\t{}".format(model_xml, model_bin))
net = IENetwork(model=model_xml, weights=model_bin)

log.info("Preparing input blobs")
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
n,c,h,w=net.inputs[input_blob].shape
print(n,c,h,w)

for i in range(n):
    np.random.seed(5)
    args.input=np.load('data2.npy')
    images = args.input.transpose((0,3, 1, 2))  # Change data layout from HWC to CHW

log.info("Batch size is {}".format(n))
# Loading model to the plugin
log.info("Loading model to the plugin")
exec_net = ie.load_network(network=net, device_name=args.device)
log.info("Starting inference in synchronous mode")
log.info("Starting inference ({} iterations)".format(args.number_iter))
infer_time = []
for i in range(args.number_iter):
    t0 = time()
    res = exec_net.infer(inputs={input_blob: images})
    infer_time.append((time() - t0) * 1000)


log.info("Processing output blob")
res = res[out_blob]
log.info("Top {} results: ".format(args.number_top))
if args.labels:
    with open(args.labels, 'r') as f:
        labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip() for x in f]
else:
    labels_map = None
    
classid_str = "classid"
probability_str = "probability"
for i, probs in enumerate(res):
    probs = np.squeeze(probs)
    top_ind = np.argsort(probs)[-args.number_top:][::-1]
    #print("Image {}\n".format(args.input[i]))
    print(classid_str, probability_str)
    print("{} {}".format('-' * len(classid_str), '-' * len(probability_str)))
    for id in top_ind:
        det_label = labels_map[id] if labels_map else "{}".format(id)
        label_length = len(det_label)
        space_num_before = (len(classid_str) - label_length) // 2
        space_num_after = len(classid_str) - (space_num_before + label_length) + 2
        space_num_before_prob = (len(probability_str) - len(str(probs[id]))) // 2
        print("{}{}{}{}{:.7f}".format(' ' * space_num_before, det_label,
                                      ' ' * space_num_after, ' ' * space_num_before_prob,
                                      probs[id]))    
if args.perf_counts:
    perf_counts = exec_net.requests[0].get_perf_counts()
    log.info("Performance counters:")
    print("{:<60} {:<15} {:<25} {:<15} {:<10}".format('name', 'layer_type', 'exet_type', 'status', 'real_time, us'))
    for layer, stats in perf_counts.items():
        layer = layer.split('/')[-1]
        print("{:<60} {:<15} {:<25} {:<15} {:<10}".format(layer, stats['layer_type'], stats['exec_type'],
                                                          stats['status'], stats['real_time']))

2020-01-13 11:24:39,221 INFO:Creating Inference Engine
2020-01-13 11:24:39,224 INFO:Loading network files:
	test_class/model_converted/model_fp16/model_fp16.xml
	test_class/model_converted/model_fp16/model_fp16.bin
2020-01-13 11:24:39,388 INFO:Preparing input blobs
1 1 3 1000
2020-01-13 11:24:39,409 INFO:Batch size is 1
2020-01-13 11:24:39,410 INFO:Loading model to the plugin
2020-01-13 11:24:41,509 INFO:Starting inference in synchronous mode
2020-01-13 11:24:41,510 INFO:Starting inference (1000 iterations)
2020-01-13 11:25:06,199 INFO:Average running time of one iteration: 24.682459831237793 ms
2020-01-13 11:25:06,199 INFO:Performance 40.51 FPS
2020-01-13 11:25:06,200 INFO:Processing output blob
2020-01-13 11:25:06,200 INFO:Top 13 results: 
classid probability
------- -----------
   3      0.9428711
  10     0.0159912
   2     0.0113297
   1     0.0066643
   5     0.0064659
   6     0.0064163
   7     0.0050583
   0     0.0040703
   9     0.0006032
   4     0.0002429
  11     0.000187

 ## model1 异步

In [1]:
import sys
import os
import numpy as np
import logging as log
from time import time
from openvino.inference_engine import IENetwork, IECore
import threading

log.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=log.INFO,stream=sys.stdout)
class InferReqWrap:
    def __init__(self, request, id, num_iter):
        self.id = id
        self.request = request
        self.num_iter = num_iter
        self.cur_iter = 0
        self.cv = threading.Condition()
        self.request.set_completion_callback(self.callback, self.id)

    def callback(self, statusCode, userdata):
        if (userdata != self.id):
            log.error("Request ID {} does not correspond to user data {}".format(self.id, userdata))
        elif statusCode != 0:
            log.error("Request {} failed with status code {}".format(self.id, statusCode))
        self.cur_iter += 1
        log.info("Completed {} Async request execution".format(self.cur_iter))
        if self.cur_iter < self.num_iter:
            # here a user can read output containing inference results and put new input
            # to repeat async request again
            self.request.async_infer(self.input)
        else:
            # continue sample execution after last Asynchronous inference request execution
            self.cv.acquire()
            self.cv.notify()
            self.cv.release()

    def execute(self, mode, input_data):
        if (mode == "async"):
            log.info("Start inference ({} Asynchronous executions)".format(self.num_iter))
            self.input = input_data
            # Start async request for the first time. Wait all repetitions of the async request
            self.request.async_infer(input_data)
            self.cv.acquire()
            self.cv.wait()
            self.cv.release()
        elif (mode == "sync"):
            infer_time = []  
    
            log.info("Start inference ({} Synchronous executions)".format(self.num_iter))
            for self.cur_iter in range(self.num_iter):
                t0 = time()
                # here we start inference synchronously and wait for
                # last inference request execution
                self.request.infer(input_data)
                infer_time.append((time() - t0) * 1000)
                log.info("Completed {} Sync request execution".format(self.cur_iter + 1))
            return infer_time
        else:
            log.error("wrong inference mode is chosen. Please use \"sync\" or \"async\" mode")
            sys.exit(1)

class ObjDict(dict):
    """Makes a dictionary behave like an object, with attribute-style access.
    """

    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)

    def __setattr__(self, name, value):
        self[name] = value
args=ObjDict()
args.device="MYRIAD"
args.number_iter=10
args.perf_counts=True
args.number_top=13
args.labels=None


if args.device in ['CPU']:
    args.model="test_class/model_converted/model_fp16/model_fp16.xml"
elif args.device in ['MYRIAD']:
    args.model="test_class/model_converted/model_fp16/model_fp16.xml"
else:
    args.model="test_class/model_converted/model_fp32/model_fp32.xml"
    #args.model="/home/ubuntu/tf_models/test20190704/model_converted/model_fp16/model_fp16.xml"
    
model_xml=args.model
model_bin = os.path.splitext(model_xml)[0] + ".bin"

log.info("Creating Inference Engine")
ie = IECore()
log.info("Loading network files:\n\t{}\n\t{}".format(model_xml, model_bin))
net = IENetwork(model=model_xml, weights=model_bin)

log.info("Preparing input blobs")
input_blob = next(iter(net.inputs))
out_blob = next(iter(net.outputs))
n,c,h,w=net.inputs[input_blob].shape
print(n,c,h,w)

for i in range(n):
    np.random.seed(5)
    args.input=np.load('data2.npy')
    images = args.input.transpose((0,3, 1, 2))  # Change data layout from HWC to CHW

log.info("Batch size is {}".format(n))
# Loading model to the plugin
log.info("Loading model to the plugin")
exec_net = ie.load_network(network=net, device_name=args.device)


# create one inference request for asynchronous execution
request_id = 0
infer_request = exec_net.requests[request_id];


request_wrap = InferReqWrap(infer_request, request_id, args.number_iter)
# Start inference request execution. Wait for last execution being completed
infer_time=request_wrap.execute("sync", {input_blob: images})

log.info("Average running time of one iteration: {} ms".format(np.average(np.asarray(infer_time))))
log.info("Performance {:.2f} FPS".format((1000/np.average(np.asarray(infer_time)))))


log.info("Processing output blob")

res = infer_request.outputs[out_blob]
log.info("Top {} results: ".format(args.number_top))
if args.labels:
    with open(args.labels, 'r') as f:
        labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip() for x in f]
else:
    labels_map = None
    
classid_str = "classid"
probability_str = "probability"
for i, probs in enumerate(res):
    probs = np.squeeze(probs)
    top_ind = np.argsort(probs)[-args.number_top:][::-1]
    #print("Image {}\n".format(args.input[i]))
    print(classid_str, probability_str)
    print("{} {}".format('-' * len(classid_str), '-' * len(probability_str)))
    for id in top_ind:
        det_label = labels_map[id] if labels_map else "{}".format(id)
        label_length = len(det_label)
        space_num_before = (len(classid_str) - label_length) // 2
        space_num_after = len(classid_str) - (space_num_before + label_length) + 2
        space_num_before_prob = (len(probability_str) - len(str(probs[id]))) // 2
        print("{}{}{}{}{:.7f}".format(' ' * space_num_before, det_label,
                                      ' ' * space_num_after, ' ' * space_num_before_prob,
                                      probs[id]))    
if args.perf_counts:
    perf_counts = exec_net.requests[0].get_perf_counts()
    log.info("Performance counters:")
    print("{:<60} {:<15} {:<25} {:<15} {:<10}".format('name', 'layer_type', 'exet_type', 'status', 'real_time, us'))
    for layer, stats in perf_counts.items():
        layer = layer.split('/')[-1]
        print("{:<60} {:<15} {:<25} {:<15} {:<10}".format(layer, stats['layer_type'], stats['exec_type'],
                                                          stats['status'], stats['real_time']))





2020-01-13 13:11:49,740 INFO:Creating Inference Engine
2020-01-13 13:11:49,741 INFO:Loading network files:
	test_class/model_converted/model_fp16/model_fp16.xml
	test_class/model_converted/model_fp16/model_fp16.bin
2020-01-13 13:11:49,749 INFO:Preparing input blobs
1 1 3 1000
2020-01-13 13:11:49,750 INFO:Batch size is 1
2020-01-13 13:11:49,751 INFO:Loading model to the plugin
2020-01-13 13:11:51,860 INFO:Start inference (10 Synchronous executions)
2020-01-13 13:11:51,885 INFO:Completed 1 Sync request execution
2020-01-13 13:11:51,910 INFO:Completed 2 Sync request execution
2020-01-13 13:11:51,934 INFO:Completed 3 Sync request execution
2020-01-13 13:11:51,959 INFO:Completed 4 Sync request execution
2020-01-13 13:11:51,984 INFO:Completed 5 Sync request execution
2020-01-13 13:11:52,008 INFO:Completed 6 Sync request execution
2020-01-13 13:11:52,033 INFO:Completed 7 Sync request execution
2020-01-13 13:11:52,058 INFO:Completed 8 Sync request execution
2020-01-13 13:11:52,082 INFO:Complet

In [7]:
import sys
import os
import pdb
import numpy as np
import logging as log
import time
from datetime import datetime
from openvino.inference_engine import IENetwork, IECore
import threading

log.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=log.INFO,stream=sys.stdout)

class InferReqWrap:
    def __init__(self, request, id, callbackQueue):
        self.id = id
        self.request = request
        self.request.set_completion_callback(self.callback, self.id)
        self.callbackQueue = callbackQueue

    def callback(self, statusCode, userdata):
        if (userdata != self.id):
            print("Request ID {} does not correspond to user data {}".format(self.id, userdata))
        elif statusCode != 0:
            print("Request {} failed with status code {}".format(self.id, statusCode))
        self.callbackQueue(self.id, self.request.latency)

    def startAsync(self, input_data):
        self.request.async_infer(input_data)

    def infer(self, input_data):
        self.request.infer(input_data)
        self.callbackQueue(self.id, self.request.latency);
class InferRequestsQueue:
    def __init__(self, requests):
        self.idleIds = []
        self.requests = []
        self.times = []
        for id in range(0, len(requests)):
            self.requests.append(InferReqWrap(requests[id], id, self.putIdleRequest))
            self.idleIds.append(id)
        self.startTime = datetime.max
        self.endTime = datetime.min
        self.cv = threading.Condition()

    def resetTimes(self):
        self.times.clear()

    def getDurationInSeconds(self):
        return (self.endTime - self.startTime).total_seconds()

    def putIdleRequest(self, id, latency):
        self.cv.acquire()
        self.times.append(latency)
        self.idleIds.append(id)
        self.endTime = max(self.endTime, datetime.now())
        self.cv.notify()
        self.cv.release()

    def getIdleRequest(self):
        self.cv.acquire()
        while len(self.idleIds) == 0:
            self.cv.wait()
        id = self.idleIds.pop();
        self.startTime = min(datetime.now(), self.startTime);
        self.cv.release()
        return self.requests[id]

    def waitAll(self):
        self.cv.acquire()
        while len(self.idleIds) != len(self.requests):
            self.cv.wait()
        self.cv.release()
def parseValuePerDevice(devices, values_string):
    ## Format: <device1>:<value1>,<device2>:<value2> or just <value>
    result = {}
    if not values_string:
        return result
    device_value_strings = values_string.upper().split(',')
    for device_value_string in device_value_strings:
        device_value_vec = device_value_string.split(':')
        if len(device_value_vec) == 2:
            for device in devices:
                if device == device_value_vec[0]:
                    value = int(device_value_vec[1])
                    result[device_value_vec[0]] = value
                    break
        elif len(device_value_vec) == 1:
            value = int(device_value_vec[0])
            for device in devices:
                result[device] = value
        elif not device_value_vec:
            raise Exception("Unknown string format: " + values_string)
    return result
def parseDevices(device_string):
    devices = device_string
    if ':' in devices:
        devices = devices.partition(':')[2]
    return [ d[:d.index('(')] if '(' in d else d for d in devices.split(',') ]
def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')
class ObjDict(dict):
    """Makes a dictionary behave like an object, with attribute-style access.
    """

    def __getattr__(self, name):
        try:
            return self[name]
        except KeyError:
            raise AttributeError(name)

    def __setattr__(self, name, value):
        self[name] = value
def test(nir=1,api_type='async'):
    args=ObjDict()
    args.device="MYRIAD"
    args.number_streams=None
    args.number_iter=10
    args.perf_counts=True
    args.number_infer_requests=nir
    args.number_top=13
    args.api_type=api_type
    args.labels=None
    args.cpu_extension=None


    if args.device in ['CPU']:
        args.model="test_class/model_converted/model_fp16/model_fp16.xml"
    elif args.device in ['MYRIAD']:
        args.model="test_class/model_converted/model_fp16/model_fp16.xml"
    else:
        args.model="test_class/model_converted/model_fp32/model_fp32.xml"
        #args.model="/home/ubuntu/tf_models/test20190704/model_converted/model_fp16/model_fp16.xml"

    model_xml = args.model
    model_bin = os.path.splitext(model_xml)[0] + ".bin"
    # Plugin initialization for specified device and load extensions library if specified
    log.info("Initializing plugin for {} device...".format(args.device))
    device_nstreams = parseValuePerDevice(args.device, args.number_streams)
    ie = IECore()
    if args.cpu_extension and 'CPU' in args.device:
        ie.add_extension(args.cpu_extension, "CPU")
    #    plugin = IEPlugin(device=args.device, plugin_dirs=args.plugin_dir)

    # Read IR
    log.info("Reading IR...")
    net = IENetwork(model=model_xml, weights=model_bin)

    if "CPU" in args.device:
        ie.set_config({'CPU_THROUGHPUT_STREAMS': str(device_nstreams.get(args.device))
                                                         if args.device in device_nstreams.keys()
                                                         else 'CPU_THROUGHPUT_AUTO' }, args.device)
        device_nstreams[args.device] = int(ie.get_config(args.device, 'CPU_THROUGHPUT_STREAMS'))
        ie.add_extension(args.cpu_extension, "CPU")
        supported_layers = ie.query_network(net, "CPU")
        not_supported_layers = [l for l in net.layers.keys() if l not in supported_layers]
        if len(not_supported_layers) != 0:
            log.error("Following layers are not supported by the plugin for specified device {}:\n {}".
                      format(args.device, ', '.join(not_supported_layers)))
            log.error("Please try to specify cpu extensions library path in sample's command line parameters using -l "
                      "or --cpu_extension command line argument")
            sys.exit(1)
    # elif "MYRIAD" in args.device:
    #     ie.set_config({'LOG_LEVEL': 'LOG_INFO',
    #                        'VPU_LOG_LEVEL': 'LOG_WARNING'}, MYRIAD_DEVICE_NAME)

    input_blob = next(iter(net.inputs))
    out_blob = next(iter(net.outputs))
    #netoutput = iter(net.outputs)
    #out_blob_loc = next(netoutput)
    #out_blob_class = next(netoutput)
    log.info("Loading IR to the plugin...")
    #    exec_net = plugin.load(network=net, num_requests=2)
    config = { 'PERF_COUNT' : ('YES' if args.perf_counts else 'NO')}

    exe_network = ie.load_network(net,
                                      args.device,
                                      config=config,
                                      num_requests=args.number_infer_requests if args.number_infer_requests else 0)
    # exec_net = ie.load_network(network=net, device_name=args.device)
    # Read and pre-process input image
    n, c, h, w = net.inputs[input_blob].shape
    del net

    if args.api_type == 'async':
        is_async_mode = True
    else:
        is_async_mode = False

    #images = np.zeros((n,c,h,w), dtype = np.float32)
    #for i in range(n):
    #    print(args.input[i])

        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #if image.shape[:-1] != (h, w):
        #    log.warning("Image {} is resized from {} to {}".format(args.input[i], image.shape[:-1], (h, w)))
        #    image = cv2.resize(image, (w, h))

    log.info("Batch size is {}".format(n))

    #frame = cv2.imread(input_stream)
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #Img = np.zeros((1,1,448,448), dtype = np.float32)
    #Img[0,0,:,:] = frame

    # create one inference request for asynchronous execution
    # create one inference request for asynchronous execution
    infer_requests = exe_network.requests
    nireq = len(infer_requests)
    print("number of infer request: ", nireq)

    images = np.zeros((nireq,n,c,h,w), dtype = np.float32)
    for i in range(nireq):
    #     args.input=np.load('data2.npy')
    #     image = args.input.transpose((0,3, 1, 2))
        image = np.random.rand(1,1,3,1000)
        images[i] = image

    request_queue = InferRequestsQueue(infer_requests)

    inf_start = time.time()
    iteration = 0

    while ((nireq and iteration < nireq) or
               (args.api_type == "async" and iteration % nireq != 0)):
        infer_request = request_queue.getIdleRequest()
        if not infer_request:
            raise Exception("No idle Infer Requests!")

        if (args.api_type == 'sync'):
            infer_request.infer({input_blob: image})
        else:
            infer_request.startAsync({input_blob: images[i]})
            print("flag")
        iteration += 1

        #exec_time = (datetime.now() - start_time).total_seconds()


        ## wait the latest inference executions
    request_queue.waitAll()
    print(iteration)
    inf_end = time.time()
    det_time = inf_end - inf_start
#     print("[Performance] inference time:{} ms".format(det_time*1000))
    return det_time*1000

        
            
# for infer_request in infer_requests:
#     log.info("Processing output blob")

#     res = infer_request.outputs[out_blob]
#     log.info("Top {} results: ".format(args.number_top))
#     if args.labels:
#         with open(args.labels, 'r') as f:
#             labels_map = [x.split(sep=' ', maxsplit=1)[-1].strip() for x in f]
#     else:
#         labels_map = None

#     classid_str = "classid"
#     probability_str = "probability"
#     for i, probs in enumerate(res):
#         probs = np.squeeze(probs)
#         top_ind = np.argsort(probs)[-args.number_top:][::-1]
#         #print("Image {}\n".format(args.input[i]))
#         print(classid_str, probability_str)
#         print("{} {}".format('-' * len(classid_str), '-' * len(probability_str)))
#         for id in top_ind:
#             det_label = labels_map[id] if labels_map else "{}".format(id)
#             label_length = len(det_label)
#             space_num_before = (len(classid_str) - label_length) // 2
#             space_num_after = len(classid_str) - (space_num_before + label_length) + 2
#             space_num_before_prob = (len(probability_str) - len(str(probs[id]))) // 2
#             print("{}{}{}{}{:.7f}".format(' ' * space_num_before, det_label,
#                                           ' ' * space_num_after, ' ' * space_num_before_prob,
#                                           probs[id]))    
# if args.perf_counts:
#         for ni in range(int(args.number_infer_requests)):
#             perf_counts = exe_network.requests[ni].get_perf_counts()
#             log.info("Pefrormance counts for {}-th infer request".format(ni))
#             for layer, stats in perf_counts.items():
#                 max_layer_name = 30
#                 print("{:<30}{:<15}{:<30}{:<20}{:<20}{:<20}".format(layer[:max_layer_name - 4] + '...' if (len(layer) >= max_layer_name) else layer,
#                                                                         stats['status'],
#                                                                         'layerType: ' + str(stats['layer_type']),
#                                                                         'realTime: ' + str(stats['real_time']),
#                                                                         'cpu: ' + str(stats['cpu_time']),
#                                                                         'execType: ' + str(stats['exec_type'])))


def test1(nir=1,api_type='async'):
    args=ObjDict()
    args.device="MYRIAD"
    args.number_streams=None
    args.number_iter=10
    args.perf_counts=True
    args.number_infer_requests=nir
    args.number_top=13
    args.api_type=api_type
    args.labels=None
    args.cpu_extension=None


    if args.device in ['CPU']:
        args.model="test/model_converted/model_fp16/model_fp16.xml"
    elif args.device in ['MYRIAD']:
        args.model="test/model_converted/model_fp16/model_fp16.xml"
    else:
        args.model="test/model_converted/model_fp32/model_fp32.xml"
        #args.model="/home/ubuntu/tf_models/test20190704/model_converted/model_fp16/model_fp16.xml"

    model_xml = args.model
    model_bin = os.path.splitext(model_xml)[0] + ".bin"
    # Plugin initialization for specified device and load extensions library if specified
    log.info("Initializing plugin for {} device...".format(args.device))
    device_nstreams = parseValuePerDevice(args.device, args.number_streams)
    ie = IECore()
    if args.cpu_extension and 'CPU' in args.device:
        ie.add_extension(args.cpu_extension, "CPU")
    #    plugin = IEPlugin(device=args.device, plugin_dirs=args.plugin_dir)

    # Read IR
    log.info("Reading IR...")
    net = IENetwork(model=model_xml, weights=model_bin)

    if "CPU" in args.device:
        ie.set_config({'CPU_THROUGHPUT_STREAMS': str(device_nstreams.get(args.device))
                                                         if args.device in device_nstreams.keys()
                                                         else 'CPU_THROUGHPUT_AUTO' }, args.device)
        device_nstreams[args.device] = int(ie.get_config(args.device, 'CPU_THROUGHPUT_STREAMS'))
        ie.add_extension(args.cpu_extension, "CPU")
        supported_layers = ie.query_network(net, "CPU")
        not_supported_layers = [l for l in net.layers.keys() if l not in supported_layers]
        if len(not_supported_layers) != 0:
            log.error("Following layers are not supported by the plugin for specified device {}:\n {}".
                      format(args.device, ', '.join(not_supported_layers)))
            log.error("Please try to specify cpu extensions library path in sample's command line parameters using -l "
                      "or --cpu_extension command line argument")
            sys.exit(1)
    # elif "MYRIAD" in args.device:
    #     ie.set_config({'LOG_LEVEL': 'LOG_INFO',
    #                        'VPU_LOG_LEVEL': 'LOG_WARNING'}, MYRIAD_DEVICE_NAME)

    input_blob = next(iter(net.inputs))
    out_blob = next(iter(net.outputs))
    #netoutput = iter(net.outputs)
    #out_blob_loc = next(netoutput)
    #out_blob_class = next(netoutput)
    log.info("Loading IR to the plugin...")
    #    exec_net = plugin.load(network=net, num_requests=2)
    config = { 'PERF_COUNT' : ('YES' if args.perf_counts else 'NO')}

    exe_network = ie.load_network(net,
                                      args.device,
                                      config=config,
                                      num_requests=args.number_infer_requests if args.number_infer_requests else 0)
    # exec_net = ie.load_network(network=net, device_name=args.device)
    # Read and pre-process input image
    n, c, h, w = net.inputs[input_blob].shape
    del net

    if args.api_type == 'async':
        is_async_mode = True
    else:
        is_async_mode = False

    #images = np.zeros((n,c,h,w), dtype = np.float32)
    #for i in range(n):
    #    print(args.input[i])

        #image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        #if image.shape[:-1] != (h, w):
        #    log.warning("Image {} is resized from {} to {}".format(args.input[i], image.shape[:-1], (h, w)))
        #    image = cv2.resize(image, (w, h))

    log.info("Batch size is {}".format(n))

    #frame = cv2.imread(input_stream)
    #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    #Img = np.zeros((1,1,448,448), dtype = np.float32)
    #Img[0,0,:,:] = frame

    # create one inference request for asynchronous execution
    # create one inference request for asynchronous execution
    infer_requests = exe_network.requests
    nireq = len(infer_requests)
    print("number of infer request: ", nireq)

    images = np.zeros((nireq,n,c,h,w), dtype = np.float32)
    for i in range(nireq):
    #     args.input=np.load('data2.npy')
    #     image = args.input.transpose((0,3, 1, 2))
        image = np.random.rand(1,1,1,500)
        images[i] = image

    request_queue = InferRequestsQueue(infer_requests)

    inf_start = time.time()
    iteration = 0

    while ((nireq and iteration < nireq) or
               (args.api_type == "async" and iteration % nireq != 0)):
        infer_request = request_queue.getIdleRequest()
        if not infer_request:
            raise Exception("No idle Infer Requests!")

        if (args.api_type == 'sync'):
            infer_request.infer({input_blob: image})
        else:
            infer_request.startAsync({input_blob: images[i]})
            print("flag")
        iteration += 1

        #exec_time = (datetime.now() - start_time).total_seconds()


        ## wait the latest inference executions
    request_queue.waitAll()
    print(iteration)
    inf_end = time.time()
    det_time = inf_end - inf_start
#     print("[Performance] inference time:{} ms".format(det_time*1000))
    return det_time*1000


In [2]:
times=[]
for i in range(1,20):
    t=test(i)
    times.append(t)
print(times)

2020-01-13 14:49:59,479 INFO:Initializing plugin for MYRIAD device...
2020-01-13 14:49:59,481 INFO:Reading IR...
2020-01-13 14:49:59,487 INFO:Loading IR to the plugin...
2020-01-13 14:50:01,607 INFO:Batch size is 1
number of infer request:  1
flag
1
2020-01-13 14:50:02,041 INFO:Initializing plugin for MYRIAD device...
2020-01-13 14:50:02,043 INFO:Reading IR...
2020-01-13 14:50:02,048 INFO:Loading IR to the plugin...
2020-01-13 14:50:04,436 INFO:Batch size is 1
number of infer request:  2
flag
flag
2
2020-01-13 14:50:04,878 INFO:Initializing plugin for MYRIAD device...
2020-01-13 14:50:04,880 INFO:Reading IR...
2020-01-13 14:50:04,886 INFO:Loading IR to the plugin...
2020-01-13 14:50:07,273 INFO:Batch size is 1
number of infer request:  3
flag
flag
flag
3
2020-01-13 14:50:07,734 INFO:Initializing plugin for MYRIAD device...
2020-01-13 14:50:07,735 INFO:Reading IR...
2020-01-13 14:50:07,740 INFO:Loading IR to the plugin...
2020-01-13 14:50:10,123 INFO:Batch size is 1
number of infer requ

In [3]:
模型2异步FPS
for i,t in enumerate(times):
    fps = (i+1)*1000/t
    print(fps)

40.89889130498376
69.14162785905626
60.613665266484254
74.09841973694671
65.6371221913761
75.500945043472
68.97116425918574
74.98532224903906
70.3182515037433
75.62482420395946
71.44182359016604
75.68942037005847
72.39873913539954
76.63378684363356
73.47265382769336
77.01782308209636
74.31539385929896
76.45483950022178
74.39187560618832


In [4]:
times=[]
for i in range(1,20):
    t=test(i,'sync')
    times.append(t)
print(times)

2020-01-13 14:58:16,485 INFO:Initializing plugin for MYRIAD device...
2020-01-13 14:58:16,491 INFO:Reading IR...
2020-01-13 14:58:16,507 INFO:Loading IR to the plugin...
2020-01-13 14:58:18,615 INFO:Batch size is 1
number of infer request:  1
1
2020-01-13 14:58:19,049 INFO:Initializing plugin for MYRIAD device...
2020-01-13 14:58:19,051 INFO:Reading IR...
2020-01-13 14:58:19,056 INFO:Loading IR to the plugin...
2020-01-13 14:58:21,440 INFO:Batch size is 1
number of infer request:  2
2
2020-01-13 14:58:21,898 INFO:Initializing plugin for MYRIAD device...
2020-01-13 14:58:21,899 INFO:Reading IR...
2020-01-13 14:58:21,904 INFO:Loading IR to the plugin...
2020-01-13 14:58:24,297 INFO:Batch size is 1
number of infer request:  3
3
2020-01-13 14:58:24,781 INFO:Initializing plugin for MYRIAD device...
2020-01-13 14:58:24,782 INFO:Reading IR...
2020-01-13 14:58:24,787 INFO:Loading IR to the plugin...
2020-01-13 14:58:27,171 INFO:Batch size is 1
number of infer request:  4
4
2020-01-13 14:58:27,

In [6]:
#模型2同步FPS
for i,t in enumerate(times):
    fps = (i+1)*1000/t
    print(fps)

40.37837785800241
40.69176813000242
40.44547874050169
40.62063522039988
40.43349586536255
40.57367835550181
41.0890337821479
40.81595833064709
40.908730919908315
40.932458531320236
40.961672795586466
40.936746542084656
40.983274756397115
41.20485502227593
41.040370726005094
41.013394513242645
41.188800378943235
41.173394879167034
41.08092436979122


In [10]:
times=[]
for i in range(1,20):
    t=test1(i)
    times.append(t)
print(times)

2020-01-13 15:37:30,231 INFO:Initializing plugin for MYRIAD device...
2020-01-13 15:37:30,237 INFO:Reading IR...
2020-01-13 15:37:30,244 INFO:Loading IR to the plugin...
2020-01-13 15:37:32,251 INFO:Batch size is 1
number of infer request:  1
flag
1
2020-01-13 15:37:32,665 INFO:Initializing plugin for MYRIAD device...
2020-01-13 15:37:32,666 INFO:Reading IR...
2020-01-13 15:37:32,669 INFO:Loading IR to the plugin...
2020-01-13 15:37:34,951 INFO:Batch size is 1
number of infer request:  2
flag
flag
2
2020-01-13 15:37:35,365 INFO:Initializing plugin for MYRIAD device...
2020-01-13 15:37:35,367 INFO:Reading IR...
2020-01-13 15:37:35,369 INFO:Loading IR to the plugin...
2020-01-13 15:37:37,645 INFO:Batch size is 1
number of infer request:  3
flag
flag
flag
3
2020-01-13 15:37:38,065 INFO:Initializing plugin for MYRIAD device...
2020-01-13 15:37:38,066 INFO:Reading IR...
2020-01-13 15:37:38,069 INFO:Loading IR to the plugin...
2020-01-13 15:37:40,343 INFO:Batch size is 1
number of infer requ

In [11]:
#模型1异步FPS
for i,t in enumerate(times):
    fps = (i+1)*1000/t
    print(fps)

196.65716429107277
257.88090626825294
265.9391736235866
258.6880888135071
269.3248744654347
274.63113439188083
273.8536903862477
273.26236236888394
273.8849136961553
278.16453891302183
277.41773795923274
279.12869002922633
279.385296468611
279.58698250208306
278.63451965490975
279.32929864724247
278.9780700896368
279.32232285562065
280.3462158149876


In [13]:
times=[]
for i in range(1,20):
    t=test1(i,'sync')
    times.append(t)
print(times)

2020-01-13 15:44:04,233 INFO:Initializing plugin for MYRIAD device...
2020-01-13 15:44:04,239 INFO:Reading IR...
2020-01-13 15:44:04,245 INFO:Loading IR to the plugin...
2020-01-13 15:44:06,253 INFO:Batch size is 1
number of infer request:  1
1
2020-01-13 15:44:06,665 INFO:Initializing plugin for MYRIAD device...
2020-01-13 15:44:06,667 INFO:Reading IR...
2020-01-13 15:44:06,669 INFO:Loading IR to the plugin...
2020-01-13 15:44:08,954 INFO:Batch size is 1
number of infer request:  2
2
2020-01-13 15:44:09,374 INFO:Initializing plugin for MYRIAD device...
2020-01-13 15:44:09,375 INFO:Reading IR...
2020-01-13 15:44:09,378 INFO:Loading IR to the plugin...
2020-01-13 15:44:11,656 INFO:Batch size is 1
number of infer request:  3
3
2020-01-13 15:44:12,082 INFO:Initializing plugin for MYRIAD device...
2020-01-13 15:44:12,084 INFO:Reading IR...
2020-01-13 15:44:12,086 INFO:Loading IR to the plugin...
2020-01-13 15:44:14,359 INFO:Batch size is 1
number of infer request:  4
4
2020-01-13 15:44:14,

In [14]:
#模型1同步FPS
for i,t in enumerate(times):
    fps = (i+1)*1000/t
    print(fps)

199.65270373191166
198.9188779018757
200.14811987020423
203.73556127653373
204.83800705208975
205.51079171940714
206.92467298150655
206.61468833319992
210.81730602762218
209.2809420452561
207.4194438824825
210.3154754194263
207.15287824448362
211.06450522986233
211.31059499219106
206.43291662565213
210.1218475756022
206.33023782064245
211.06881765427227
